In [ ]:
#####################################################################
#
#       aiplatform.CustomJob.from_local_script
#
#####################################################################

In [ ]:
#####################################################################
#
# define the training script
#
#####################################################################

In [ ]:
%%writefile training_script.py

import argparse
import pickle
import os

# Sample Decision Tree Classifier
from sklearn import datasets
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

from google.cloud import storage

# parse args
parser = argparse.ArgumentParser()
parser.add_argument('--bucket_name', dest='bucket_name', default="", type=str, help='The GCS bucket to store model artifacts -> w/o gs://')
parser.add_argument('--max_depth', dest='max_depth', default=10, type=int, help='The maximum depth of the tree')
args = parser.parse_args()

# load the iris datasets
dataset = datasets.load_iris()

# fit a CART model to the data
model = DecisionTreeClassifier(max_depth = args.max_depth)
model.fit(dataset.data, dataset.target)
print(model)

# make predictions
expected = dataset.target
predicted = model.predict(dataset.data)

# summarize the fit of the model
classification_report = metrics.classification_report(expected, predicted)
confusion_matrix = metrics.confusion_matrix(expected, predicted)

# save the model to disk
model_filename = "model.pkl"
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)

# Upload the saved model file to GCS
storage_client = storage.Client()
bucket = storage_client.get_bucket(args.bucket_name)
model_directory = os.environ["AIP_MODEL_DIR"]
storage_path = os.path.join(model_directory, model_filename)
blob = storage.blob.Blob.from_string(storage_path, client=storage_client)
blob.upload_from_filename(model_filename)

In [ ]:
#####################################################################
#
# kick off the custom training job
#
#####################################################################

In [ ]:
from google.cloud import aiplatform
from datetime import datetime

In [ ]:
# specify parameters
P = ! gcloud config list --format 'value(core.project)'
PROJECT_ID = P[0]
REGION = "us-central1"
BUCKET_NAME = f"bkt-{PROJECT_ID}-vpipelines"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_PATH}/pipeline_root"
PIPELINE_DATA = f"{BUCKET_PATH}/data"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
MAX_DEPTH = 20

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_PATH)

job = aiplatform.CustomJob.from_local_script(
    display_name = f"vai_CustomJob_fromLocalScript_{TIMESTAMP}"
    , project = PROJECT_ID
    , location = REGION
    , script_path = "training_script.py"
    , container_uri = "us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest"
    , requirements = ["gcsfs"]
    , replica_count = 1
    , machine_type = "n1-standard-4"
    , accelerator_count = 0
    , args = [f"--bucket_name={BUCKET_NAME}", f"--max_depth={str(MAX_DEPTH)}"]
    , environment_variables = { 'MY_KEY': 'MY_VALUE' }
    , labels={'my_key': 'my_value'}
)

In [ ]:
job.run(service_account = f"sa-vertex-pipelines@{PROJECT_ID}.iam.gserviceaccount.com")